In [ ]:
#coding=utf-8
import requests
import re
import os
import time
from bs4 import BeautifulSoup
import sys
import threading

def new_thread(url):
	thread1=download(url)
	thread1.start()

class download (threading.Thread):
	def __init__(self,url):
		threading.Thread.__init__(self)
		self.url = url
        
	def run(self):
		url = self.url
		
		
		
		#os.system('start python 18comic.py')
		headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}
		r_url_x=requests.get(url,headers=headers)
		p_image_link = re.compile(r'/photo/\d+/')
		res_link = []
		res_link = p_image_link.findall(r_url_x.text)
		
		p_split_123 = re.compile(r'/album/')

		url=p_split_123.split(url)[0]+res_link[0]
		
		with open('log','a+') as f:
			f.write('\n-----------------------\n'+'\nStart downloading:'+url+'\n-----------------------\n')

		r_url=requests.get(url,headers=headers)
		
		soup = BeautifulSoup(r_url.text,'lxml')
		
		p_image_link = re.compile(r'/media/photos/\S+/\S+jpg')
		res_images_link = []
		res_images_link = p_image_link.findall(r_url.text)
		
		p_split_name = re.compile(r'/')
		
		
		foldername = soup.title.string
		
		foldername = foldername.strip('\n')
		foldername = foldername.replace('?','')
		foldername = foldername.replace('\\','')
		foldername = foldername.replace('/','')
		foldername = foldername.replace(':','')
		foldername = foldername.replace('*','')
		foldername = foldername.replace('<','')
		foldername = foldername.replace('>','')
		foldername = foldername.replace('|','')
		foldername = foldername.replace('"','')
		foldername = foldername.replace('H漫內頁瀏覽 Comics - 禁漫天堂','')
		current_dir = os.path.dirname(os.path.abspath(__file__))
		comicPath = current_dir + '\\comic\\'
		FilePath = comicPath+foldername+'\\'
		
		
		
		if os.path.exists(comicPath) == False:
			os.mkdir(comicPath)
		
		if os.path.exists(FilePath) == False:
			os.mkdir(FilePath)
			
		imageNumMax = len(res_images_link)
		imageNumMin = 0
		p_split_18comic = re.compile(r'/photo/')
		for imageLink in res_images_link:
			r_imageLink=requests.get(p_split_18comic.split(url)[0]+imageLink,headers=headers)
			filename = p_split_name.split(imageLink)[4]
			with open(FilePath+filename,'wb+') as f:
				f.write(r_imageLink.content)
			imageNumMin=imageNumMin+1
			#sys.stdout.write('\r'+str(imageNumMin)+'/'+str(imageNumMax)+' '+str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())))
			#sys.stdout.flush()
			
		f = open ('log', "r+")
		open('log_', 'w+').write(re.sub('\n-----------------------\n'+'\nStart downloading:'+url+'\n-----------------------\n','',f.read()))
		f.close()
		os.system('del /q log')
		os.rename('log_','log')
		with open('log','a+') as f:
			f.write('\n-----------------------\n'+'\nFinished downloading:'+url+'\n-----------------------\n')
		print('\n'+foldername+' 下载完成!! 已保存到: '+FilePath)
		
		
		
print('输入18comic的漫画页url,然后提取该漫画的所有图片保存到文件夹下')
print('url(例如: https://18comic.org/album/125840/jinani-sound-team%E6%B1%89%E5%8C%96-c89-trick-treat-tries-trick-effect-2-sound-voltex)')
print('输入后按 enter 会新建一个线程开始下载, 然后你可以继续输入新的链接继续爬本子')
print('--------------------------------------------------')


while True:
	url = input('url=')
	new_thread(url)
